In [1]:
import pandas as pd
import scipy.stats as st
from datetime import datetime
from datetime import timedelta
import math
import numpy as np
from PIL import Image
from scipy import stats
from keras.utils.np_utils import to_categorical
from devol import DEvol, GenomeHandler
import tensorflow as tf
import keras.backend.tensorflow_backend as K
from sklearn.model_selection import train_test_split


import os, glob, numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
patient_route = pd.read_csv('covid19/PatientRoute.csv')
patient_info = pd.read_csv('covid19/PatientInfo.csv')

dates = patient_route['date']
first = datetime.strptime(dates[0], '%Y-%m-%d')
last = datetime.strptime(dates.max(), '%Y-%m-%d')

In [3]:
patient_route

,patient_id,global_num,date,province,city,type,latitude,longitude
0,1000000001,2.0,2020-01-22,Gyeonggi-do,Gimpo-si,airport,37.615246,126.715632
1,1000000001,2.0,2020-01-24,Seoul,Jung-gu,hospital,37.567241,127.005659
2,1000000002,5.0,2020-01-25,Seoul,Seongbuk-gu,etc,37.592560,127.017048
3,1000000002,5.0,2020-01-26,Seoul,Seongbuk-gu,store,37.591810,127.016822
4,1000000002,5.0,2020-01-26,Seoul,Seongdong-gu,public_transportation,37.563992,127.029534
...,...,...,...,...,...,...,...,...
6709,6100000090,NaN,2020-03-24,Seoul,Gangseo-gu,airport,37.558654,126.794474
6710,6100000090,NaN,2020-03-24,Busan,Gangseo-gu,airport,35.173220,128.946459
6711,6100000090,NaN,2020-03-25,Gyeongsangnam-do,Yangsan-si,store,35.336944,129.026389
6712,6100000090,NaN,2020-03-25,Gyeongsangnam-do,Yangsan-si,hospital,35.335757,129.025003


In [4]:
patient_info

,patient_id,global_num,sex,birth_year,age,country,province,city,disease,infection_case,infection_order,infected_by,contact_number,symptom_onset_date,confirmed_date,released_date,deceased_date,state
0,1000000001,2.0,male,1964.0,50s,Korea,Seoul,Gangseo-gu,NaN,overseas inflow,1.0,NaN,75.0,2020-01-22,2020-01-23,2020-02-05,NaN,released
1,1000000002,5.0,male,1987.0,30s,Korea,Seoul,Jungnang-gu,NaN,overseas inflow,1.0,NaN,31.0,NaN,2020-01-30,2020-03-02,NaN,released
2,1000000003,6.0,male,1964.0,50s,Korea,Seoul,Jongno-gu,NaN,contact with patient,2.0,2.002000e+09,17.0,NaN,2020-01-30,2020-02-19,NaN,released
3,1000000004,7.0,male,1991.0,20s,Korea,Seoul,Mapo-gu,NaN,overseas inflow,1.0,NaN,9.0,2020-01-26,2020-01-30,2020-02-15,NaN,released
4,1000000005,9.0,female,1992.0,20s,Korea,Seoul,Seongbuk-gu,NaN,contact with patient,2.0,1.000000e+09,2.0,NaN,2020-01-31,2020-02-24,NaN,released
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3383,7000000009,9651.0,female,NaN,20s,Korea,Jeju-do,Jeju-do,NaN,overseas inflow,NaN,NaN,14.0,NaN,2020-03-29,NaN,NaN,isolated
3384,7000000010,NaN,female,NaN,20s,Korea,Jeju-do,Jeju-do,NaN,overseas inflow,NaN,NaN,18.0,NaN,2020-04-03,NaN,NaN,isolated
3385,7000000011,NaN,male,NaN,30s,Korea,Jeju-do,Jeju-do,NaN,contact with patient,NaN,7.000000e+09,5.0,NaN,2020-04-03,NaN,NaN,isolated
3386,7000000012,NaN,female,NaN,20s,Korea,Jeju-do,Jeju-do,NaN,overseas inflow,NaN,NaN,9.0,NaN,2020-04-03,NaN,NaN,isolated


In [5]:
infected_by = patient_info['infected_by'].dropna()
infected_by = infected_by.drop_duplicates(keep='first', inplace=False)#.tolist()
print(infected_by)


2       2.002000e+09
4       1.000000e+09
5       1.000000e+09
12      1.000000e+09
13      1.000000e+09
            ...     
3337    6.100000e+09
3347    6.002000e+09
3369    6.100000e+09
3370    6.100000e+09
3385    7.000000e+09
Name: infected_by, Length: 370, dtype: float64


In [6]:
patient_info = patient_info[["patient_id","global_num","sex","age","infection_case"]]

In [7]:
merge_route = pd.merge(patient_info, patient_route)
merge_route

,patient_id,global_num,sex,age,infection_case,date,province,city,type,latitude,longitude
0,1000000001,2.0,male,50s,overseas inflow,2020-01-22,Gyeonggi-do,Gimpo-si,airport,37.615246,126.715632
1,1000000001,2.0,male,50s,overseas inflow,2020-01-24,Seoul,Jung-gu,hospital,37.567241,127.005659
2,1000000002,5.0,male,30s,overseas inflow,2020-01-25,Seoul,Seongbuk-gu,etc,37.592560,127.017048
3,1000000002,5.0,male,30s,overseas inflow,2020-01-26,Seoul,Seongbuk-gu,store,37.591810,127.016822
4,1000000002,5.0,male,30s,overseas inflow,2020-01-26,Seoul,Seongdong-gu,public_transportation,37.563992,127.029534
...,...,...,...,...,...,...,...,...,...,...,...
6709,6100000090,NaN,male,40s,overseas inflow,2020-03-24,Seoul,Gangseo-gu,airport,37.558654,126.794474
6710,6100000090,NaN,male,40s,overseas inflow,2020-03-24,Busan,Gangseo-gu,airport,35.173220,128.946459
6711,6100000090,NaN,male,40s,overseas inflow,2020-03-25,Gyeongsangnam-do,Yangsan-si,store,35.336944,129.026389
6712,6100000090,NaN,male,40s,overseas inflow,2020-03-25,Gyeongsangnam-do,Yangsan-si,hospital,35.335757,129.025003


In [8]:
merge_route = merge_route[merge_route["province"]=="Seoul"]
merge_route

,patient_id,global_num,sex,age,infection_case,date,province,city,type,latitude,longitude
1,1000000001,2.0,male,50s,overseas inflow,2020-01-24,Seoul,Jung-gu,hospital,37.567241,127.005659
2,1000000002,5.0,male,30s,overseas inflow,2020-01-25,Seoul,Seongbuk-gu,etc,37.592560,127.017048
3,1000000002,5.0,male,30s,overseas inflow,2020-01-26,Seoul,Seongbuk-gu,store,37.591810,127.016822
4,1000000002,5.0,male,30s,overseas inflow,2020-01-26,Seoul,Seongdong-gu,public_transportation,37.563992,127.029534
5,1000000002,5.0,male,30s,overseas inflow,2020-01-26,Seoul,Seongbuk-gu,public_transportation,37.590330,127.015221
...,...,...,...,...,...,...,...,...,...,...,...
5875,6011000010,NaN,female,20s,contact with patient,2020-02-19,Seoul,Mapo-gu,store,37.551936,126.921668
6298,6023000011,839.0,female,10s,NaN,2020-02-21,Seoul,Dongjak-gu,hospital,37.498990,126.937719
6492,6100000024,NaN,male,30s,contact with patient,2020-02-24,Seoul,Gangdong-gu,pharmacy,37.533819,127.137649
6708,6100000090,NaN,male,40s,overseas inflow,2020-03-24,Seoul,Gangseo-gu,airport,37.558654,126.794474


In [9]:
merge_route = merge_route[pd.notnull(merge_route['sex'])]
merge_route = merge_route[pd.notnull(merge_route['age'])]

In [10]:
class GridMap:
    Re = 6371.00877  ##  지도반경
    grid = 0.16  ##  격자간격 (km)
    slat1 = 30.0  ##  표준위도 1
    slat2 = 60.0  ##  표준위도 2
    olon = 127.0175205  ##  기준점 경도
    olat = 37.523124  ##  기준점 위도
    xo = 18.4 / grid #  기준점 X좌표
    yo = 21.6 / grid  ##  기준점 Y좌표
    row = 256
    column = 256
    seoul_start_lat = 37.698098
    seoul_start_lon = 126.799791
    seoul_end_lat = 37.34815
    seoul_end_lon = 127.23525

    def __init__(self):
        self.PI = math.asin(1.0) * 2.0
        self.DEGRAD = self.PI / 180.0
        self.RADDEG = 180.0 / self.PI

        self.re = self.Re / self.grid
        self.slat1 = self.slat1 * self.DEGRAD
        self.slat2 = self.slat2 * self.DEGRAD
        self.olon = self.olon * self.DEGRAD
        self.olat = self.olat * self.DEGRAD

        self.sn = math.tan(self.PI * 0.25 + self.slat2 * 0.5) / \
                  math.tan(self.PI * 0.25 + self.slat1 * 0.5)
        self.sn = math.log(math.cos(self.slat1) /
                           math.cos(self.slat2)) / math.log(self.sn)
        self.sf = math.tan(self.PI * 0.25 + self.slat1 * 0.5)
        self.sf = math.pow(self.sf, self.sn) * math.cos(self.slat1) / self.sn
        self.ro = math.tan(self.PI * 0.25 + self.olat * 0.5)
        self.ro = self.re * self.sf / math.pow(self.ro, self.sn)

    def toGrid(self, lat, lon):
        ra = math.tan(self.PI * 0.25 + lat * self.DEGRAD * 0.5)
        ra = self.re * self.sf / pow(ra, self.sn)
        theta = lon * self.DEGRAD - self.olon
        if theta > self.PI:
            theta -= 2.0 * self.PI
        if theta < self.PI * -1:
            theta += 2.0 * self.PI
        theta *= self.sn
        x = (ra * math.sin(theta)) + self.xo
        y = (self.ro - ra * math.cos(theta)) + self.yo
        x = int(x + 1.5)
        # y = int(y + 1.5)
        y = (int(y + 1.5) - 255) * -1
        return x, y

    def toCoordinate(self, x, y):
        x = x - 1
        y = y - 1
        xn = x - self.xo
        yn = self.ro - y + self.yo
        ra = math.sqrt(xn * xn + yn * yn)
        if self.sn < 0.0:
            ra = -ra
        alat = math.pow((self.re * self.sf / ra), (1.0 / self.sn))
        alat = 2.0 * math.atan(alat) - self.PI * 0.5
        if math.fabs(xn) <= 0.0:
            theta = 0.0
        else:
            if math.fabs(yn) <= 0.0:
                theta = self.PI * 0.5
                if xn < 0.0:
                    theta = -theta
            else:
                theta = math.atan2(xn, yn)
        alon = theta / self.sn + self.olon
        lat = alat * self.RADDEG
        lon = alon * self.RADDEG
        return lat, lon

    def isSeoul(self, lat, lon):
        if self.seoul_end_lat > lat or lat > self.seoul_start_lat:
            return False
        if self.seoul_end_lon < lon or lon < self.seoul_start_lon:
            return False
        return True

    def generateMap(self, raw_data):
        gridx, gridy = [], []
        for idx, data in raw_data.iterrows():
            try:
                float(data.latitude)
            except ValueError:
                continue
            if self.isSeoul(data.latitude, data.longitude):
                x, y = self.toGrid(data.latitude, data.longitude)
                gridx.append(x), gridy.append(y)

        df = pd.DataFrame({'gridx': gridx, 'gridy': gridy})

        grid_array = np.zeros((self.row, self.column))
        for idx, data in df.iterrows():
            try:
                grid_array[data.gridy, data.gridx] += 1
            except:
                pass

        return grid_array

In [11]:
converter = GridMap()

In [12]:
merge_route.assign(Name='contact')
merge_route

,patient_id,global_num,sex,age,infection_case,date,province,city,type,latitude,longitude
1,1000000001,2.0,male,50s,overseas inflow,2020-01-24,Seoul,Jung-gu,hospital,37.567241,127.005659
2,1000000002,5.0,male,30s,overseas inflow,2020-01-25,Seoul,Seongbuk-gu,etc,37.592560,127.017048
3,1000000002,5.0,male,30s,overseas inflow,2020-01-26,Seoul,Seongbuk-gu,store,37.591810,127.016822
4,1000000002,5.0,male,30s,overseas inflow,2020-01-26,Seoul,Seongdong-gu,public_transportation,37.563992,127.029534
5,1000000002,5.0,male,30s,overseas inflow,2020-01-26,Seoul,Seongbuk-gu,public_transportation,37.590330,127.015221
...,...,...,...,...,...,...,...,...,...,...,...
5875,6011000010,NaN,female,20s,contact with patient,2020-02-19,Seoul,Mapo-gu,store,37.551936,126.921668
6298,6023000011,839.0,female,10s,NaN,2020-02-21,Seoul,Dongjak-gu,hospital,37.498990,126.937719
6492,6100000024,NaN,male,30s,contact with patient,2020-02-24,Seoul,Gangdong-gu,pharmacy,37.533819,127.137649
6708,6100000090,NaN,male,40s,overseas inflow,2020-03-24,Seoul,Gangseo-gu,airport,37.558654,126.794474


In [13]:
for index, row in merge_route.iterrows():
    if any(infected_by == row['patient_id']):
        merge_route.at[index,'contact'] = 1

In [14]:
merge_route

,patient_id,global_num,sex,age,infection_case,date,province,city,type,latitude,longitude,contact
1,1000000001,2.0,male,50s,overseas inflow,2020-01-24,Seoul,Jung-gu,hospital,37.567241,127.005659,NaN
2,1000000002,5.0,male,30s,overseas inflow,2020-01-25,Seoul,Seongbuk-gu,etc,37.592560,127.017048,1.0
3,1000000002,5.0,male,30s,overseas inflow,2020-01-26,Seoul,Seongbuk-gu,store,37.591810,127.016822,1.0
4,1000000002,5.0,male,30s,overseas inflow,2020-01-26,Seoul,Seongdong-gu,public_transportation,37.563992,127.029534,1.0
5,1000000002,5.0,male,30s,overseas inflow,2020-01-26,Seoul,Seongbuk-gu,public_transportation,37.590330,127.015221,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5875,6011000010,NaN,female,20s,contact with patient,2020-02-19,Seoul,Mapo-gu,store,37.551936,126.921668,NaN
6298,6023000011,839.0,female,10s,NaN,2020-02-21,Seoul,Dongjak-gu,hospital,37.498990,126.937719,NaN
6492,6100000024,NaN,male,30s,contact with patient,2020-02-24,Seoul,Gangdong-gu,pharmacy,37.533819,127.137649,NaN
6708,6100000090,NaN,male,40s,overseas inflow,2020-03-24,Seoul,Gangseo-gu,airport,37.558654,126.794474,NaN


In [15]:
# 환자별로 일단 나눠야 함
patients = merge_route['patient_id']
patients = patients.drop_duplicates(keep='last')
patients = patients.tolist()
type(patients)

list

In [16]:
def day_weight(day):
    if day == 0: return 252
    
    return 36 * day

In [17]:
def age_weight(age):
    age = int(age[:-1])
    
    if age == 0: return 23 * 3
    elif age == 80: return 23 * 2
    elif age == 90: return 23 * 1
    elif age == 100: return 23 * 0
    elif age == 10: return 23 * 7
    elif age == 20: return 23 * 10
    elif age == 30: return 23 * 9
    elif age == 40: return 23 * 8
    elif age == 50: return 23 * 6
    elif age == 60: return 23 * 5
    elif age == 70: return 23 * 4
    
    return 0

In [18]:
def sex_weight(sex):
    if sex == "male": return 255
    return 0

In [19]:
def infection_case_weight(infection):
    six = ['bar', 'church', 'karaoke', 'public-transportation']
    five = ['bakery', 'banc', 'etc', 'lodging', 'pharmacy', 'post-office', 'real_estate_agency', 'store']
    four = ['gym', 'pc-cafe', 'school']
    three = ['airport', 'restaurant']
    two = []
    one = ['gas_station']
    
    if infection in six: return 85 * 6
    elif infection in five: return 85 * 5
    elif infection in four: return 85 * 4
    elif infection in three: return 85 * 3
    elif infection in two: return 85 * 2
    return 85

In [20]:
def infection_type_weight(infection_type):
    if infection_type is 'overseas inflow': return 64
    elif infection_type is 'etc': return 64 * 2
    elif infection_type is 'contact with patient': return 64 * 3
    return 64* 4

In [21]:
def contact_weight(contact):
    if contact == 1: return 255
    return 0

In [22]:
class Route:
    def __init__(self, weekday, age, sex, contact, infection_case, visit_type, row, column):
        self.weekday = weekday
        self.age = age
        self.sex = sex
        self.contact = contact
        self.infection_case = infection_case
        self.visit_type = visit_type
        self.row = row
        self.column = column
        self.test = contact
        
    def day_weight(self):
        if self.weekday == 0: return 252
        return 36 * self.weekday
    
    def age_weight(self):
        age = int(self.age[:-1])

        if age == 0: return 23 * 3
        elif age == 80: return 23 * 2
        elif age == 90: return 23 * 1
        elif age == 100: return 23 * 0
        elif age == 10: return 23 * 7
        elif age == 20: return 23 * 10
        elif age == 30: return 23 * 9
        elif age == 40: return 23 * 8
        elif age == 50: return 23 * 6
        elif age == 60: return 23 * 5
        elif age == 70: return 23 * 4

        return 0
    
    def sex_weight(self):
        if self.sex == "male": return 255
        return 0
    
    def visit_type_weight(self):
        six = ['bar', 'church', 'karaoke', 'public-transportation']
        five = ['bakery', 'banc', 'etc', 'lodging', 'pharmacy', 'post-office', 'real_estate_agency', 'store']
        four = ['gym', 'pc-cafe', 'school']
        three = ['airport', 'restaurant']
        two = []
        one = ['gas_station']

        if self.visit_type in six: return 85 * 6
        elif self.visit_type in five: return 85 * 5
        elif self.visit_type in four: return 85 * 4
        elif self.visit_type in three: return 85 * 3
        elif self.visit_type in two: return 85 * 2
        return 85
    
    def infection_case_weight(self):
        if self.infection_case is 'overseas inflow': return 64
        elif self.infection_case is 'etc': return 64 * 2
        elif self.infection_case is 'contact with patient': return 64 * 3
        return 64* 4
    
    def contact_weight(self):
        if self.contact == 1: return 255
        return 0
    
    def get_image1(self, day_r, age_g, sex_b):
        day = self.day_weight()
        age = self.age_weight()
        sex = self.sex_weight()
        
        day_r[self.row][self.column] = day if day > day_r[self.row][self.column] else day_r[self.row][self.column]
        age_g[self.row][self.column] = age if age > age_g[self.row][self.column] else age_g[self.row][self.column]
        sex_b[self.row][self.column] = sex if sex > sex_b[self.row][self.column] else sex_b[self.row][self.column]
             
        return day_r, age_g, sex_b
    
    def get_image2(self, contact_r, visit_type_g, infection_case_b):
        contact = self.contact_weight()
        visit_type = self.visit_type_weight()
        infection_case = self.infection_case_weight()
        
        contact_r[self.row][self.column] = contact if contact > contact_r[self.row][self.column] else contact_r[self.row][self.column]
        visit_type_g[self.row][self.column] = visit_type if visit_type > visit_type_g[self.row][self.column] else visit_type_g[self.row][self.column]
        infection_case_b[self.row][self.column] = infection_case if infection_case > infection_case_b[self.row][self.column] else infection_case_b[self.row][self.column]
             
        return contact_r, visit_type_g, infection_case_b

In [23]:
images = []
for i in range(len(patients)):
    # 환자들 동선 가져오기
    route = merge_route.loc[merge_route['patient_id'] == patients[i]]
    
    # 날짜
    dates = route['date']
    last = datetime.strptime(dates.max(), '%Y-%m-%d')
    dates = dates.tolist()
    first = datetime.strptime(dates[0], '%Y-%m-%d')
    duration = last - first
    days = duration.days
    
    day = first
    # 날짜 순서에 따른 동선
    for j in range(0, days):
        # 1. 첫 날짜 요일/나이대/성별/방문목적/감염 원인/접촉 유무/row/column 구하기
        route_day = route.loc[route['date'] == str(day.strftime("%Y-%m-%d"))]
        
        day_route = []
        for single_route in route_day.iterrows():
            weekday = single_route[1]['date']
            weekday = datetime.strptime(weekday, "%Y-%m-%d")
            weekday = weekday.weekday()
            age = single_route[1]['age']
            sex = single_route[1]['sex']
            contact = single_route[1]['contact']
            infection_case = single_route[1]['infection_case']
            visit_type = single_route[1]['type']
            lat = single_route[1]['latitude']
            lon = single_route[1]['longitude']
            row, column = converter.toGrid(lat, lon)
            if row < 0 or row > 512 or column < 0 or column > 512: continue 
            one_route = Route(weekday, age, sex, contact, infection_case, visit_type, row, column)
            day_route.append(one_route)
        
        
        # 2. 두번째 날
        day_2 = day + timedelta(days=1)
        route_day = route.loc[route['date'] == str(day_2.strftime("%Y-%m-%d"))]
        for single_route in route_day.iterrows():
            weekday = single_route[1]['date']
            weekday = datetime.strptime(weekday, "%Y-%m-%d")
            weekday = weekday.weekday()
            age = single_route[1]['age']
            sex = single_route[1]['sex']
            contact = single_route[1]['contact']
            infection_case = single_route[1]['infection_case']
            visit_type = single_route[1]['type']
            lat = single_route[1]['latitude']
            lon = single_route[1]['longitude']
            row, column = converter.toGrid(lat, lon)
            if row < 0 or row > 512 or column < 0 or column > 512: continue 
            one_route = Route(weekday, age, sex, contact, infection_case, visit_type, row, column)
            day_route.append(one_route)
            
        # 3. 세번째 날
        day_3 = day + timedelta(days=1)
        route_day = route.loc[route['date'] == str(day_3.strftime("%Y-%m-%d"))]
        for single_route in route_day.iterrows():
            weekday = single_route[1]['date']
            weekday = datetime.strptime(weekday, "%Y-%m-%d")
            weekday = weekday.weekday()
            age = single_route[1]['age']
            sex = single_route[1]['sex']
            contact = single_route[1]['contact']
            infection_case = single_route[1]['infection_case']
            visit_type = single_route[1]['type']
            lat = single_route[1]['latitude']
            lon = single_route[1]['longitude']
            row, column = converter.toGrid(lat, lon)
            if row < 0 or row > 512 or column < 0 or column > 512: continue 
            one_route = Route(weekday, age, sex, contact, infection_case, visit_type, row, column)
            day_route.append(one_route)
        
        images.append(day_route)
        day = day + timedelta(days=1)
        
print(len(images))

1842


In [24]:
def to_image(r, g, b):
    arr = np.zeros((256,256, 3))
    arr[:,:,0] = r
    arr[:,:,1] = g
    arr[:,:,2] = b
    img = Image.fromarray(arr.astype('uint8'), 'RGB')
    return arr

def gray_image(elem):
    img = Image.fromarray(elem.astype('uint8'), 'L')
    return img

In [ ]:
def to_6_image(a, b, c, d, e, f):
    arr = np.zeros((256,256, 6))
    arr[:,:,0] = a
    arr[:,:,1] = b
    arr[:,:,2] = c
    arr[:,:,3] = d
    arr[:,:,4] = e
    arr[:,:,5] = f
    return arr

In [25]:
dataset1 = []
dataset2 = []
X1 = []
y1 = []
X2 = []
y2 = []
i = 0
for image in images:
    r1 = np.zeros((256,256))
    g1 = np.zeros((256,256))
    b1 = np.zeros((256,256))
    
    r2 = np.zeros((256,256))
    g2 = np.zeros((256,256))
    b2 = np.zeros((256,256))
    
    test1 = 0
    test2 = 0
    for day_route in image:
        r1, g1, b1 = day_route.get_image1(r1, g1, b1)
        r2, g2, b2 = day_route.get_image2(r2, g2, b2)
        test1 = 1 if day_route.test == 1 else test1
        test2 = 1 if day_route.test == 1 else test2
        
    if test1 == 0:
        test1 = [1, 0]
    else:
        test1 = [0, 1]
        
    if test2 == 0:
        test2 = [1, 0]
    else:
        test2 = [0, 1]
    
    img = to_6_image(r1,g1,b1,r2,g2,b2)
    
    X1.append(img)
    y1.append(test1)
    
    dataset1.append([img1, test1])
    dataset2.append([img2, test2])
print(len(dataset1))
print(len(dataset2))

X1 = np.array(X1)
y1 = np.array(y1)

X_train, X_test, y_train, y_test = train_test_split(X1, y1)

1842
1842


In [27]:
X_train.shape[0]

1381

In [32]:
X_test.shape

(461, 256, 256, 3)

In [28]:
from keras.datasets import mnist
from keras.utils.np_utils import to_categorical
import numpy as np
from keras import backend as K
from devol import DEvol, GenomeHandler

In [33]:
K.set_image_data_format("channels_last")

X_train = X_train.reshape(X_train.shape[0], 256, 256, 6).astype('float32') / 255
X_test = X_test.reshape(X_test.shape[0], 256, 256, 6).astype('float32') / 255
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
dataset = ((X_train, y_train), (X_test, y_test))

# **Prepare the genome configuration**
# The `GenomeHandler` class handles the constraints that are imposed upon
# models in a particular genetic program. See `genome-handler.py`
# for more information.

genome_handler = GenomeHandler(max_conv_layers=6, 
                               max_dense_layers=2, # includes final dense layer
                               max_filters=256,
                               max_dense_nodes=1024,
                               input_shape=X_train.shape[1:],
                               n_classes=10)

# **Create and run the genetic program**
# The next, and final, step is create a `DEvol` and run it. Here we specify
# a few settings pertaining to the genetic program. The program
# will save each genome's encoding, as well as the model's loss and
# accuracy, in a `.csv` file printed at the beginning of program.
# The best model is returned decoded and with `epochs` training done.

devol = DEvol(genome_handler)
model = devol.run(dataset=dataset,
                  num_generations=10,
                  pop_size=10,
                  epochs=5)
print(model.summary())

Genome encoding and metric data stored at Fri May  8 01:24:00 2020.csv 


model 1/10 - generation 1/10:

An error occurred and the model could not train:
Error when checking target: expected dense_2 to have 2 dimensions, but got array with shape (1381, 2, 2, 2, 2)
Model assigned poor score. Please ensure that your modelconstraints live within your computational resources.

model 2/10 - generation 1/10:

An error occurred and the model could not train:
Error when checking target: expected dense_2 to have 2 dimensions, but got array with shape (1381, 2, 2, 2, 2)
Model assigned poor score. Please ensure that your modelconstraints live within your computational resources.

model 3/10 - generation 1/10:

An error occurred and the model could not train:
Error when checking target: expected dense_1 to have 2 dimensions, but got array with shape (1381, 2, 2, 2, 2)
Model assigned poor score. Please ensure that your modelconstraints live within your computational resources.

model 4/10 - generat

ResourceExhaustedError: OOM when allocating tensor with shape[4194304,1024] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:RandomUniform]